
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190531
last_week=20190524
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].dropna().sum()/len(df['week_return'].dropna()),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')

print(f'Total stock number = {len(df)}')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

# calculate return & risk rank
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

# last week performance
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)

# drop used columns
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean'], inplace=True)

print(f'Total stock number after bottom volume & price removed = {len(df)}')
print('Price 70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1356
Total stock number after bottom volume & price removed = 878
Price 70% pricetile = 2.03
Sim return 90% percentile = 1.1693000000000002


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FTC,2019-05-31,0.155,0.024874,0.190538,5.325,100.000,100.000,71.640,NaN,NaN,NaN
ANO,2019-05-31,6.760,0.021026,0.066211,4.132,99.886,78.246,21.526,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,0.019912,0.106839,4.031,99.772,99.203,22.779,NaN,NaN,NaN
FEX,2019-05-31,0.105,0.020393,0.091461,3.991,99.658,95.900,98.405,3.483,0.508,10.476
AVH,2019-05-31,0.450,0.019414,0.075731,3.937,99.544,87.472,98.633,3.976,-0.039,-1.111
8CO,2019-05-31,0.155,0.017824,0.076189,3.486,99.431,88.383,44.761,3.636,-0.150,0.000
LTR,2019-05-31,0.105,0.018377,0.094203,3.442,99.317,97.608,98.975,3.816,-0.374,-4.762
WZR,2019-05-31,0.150,0.017412,0.100354,3.375,99.203,98.405,96.014,3.627,-0.252,-13.333
ISX,2019-05-31,0.620,0.015591,0.056436,3.123,99.089,65.945,85.649,3.180,-0.057,-6.452


In [2]:
# high mean return 
df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FTC,2019-05-31,0.155,0.024874,0.190538,5.325,100.000,100.000,71.640,NaN,NaN,NaN
ANO,2019-05-31,6.760,0.021026,0.066211,4.132,99.886,78.246,21.526,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,0.019912,0.106839,4.031,99.772,99.203,22.779,NaN,NaN,NaN
FEX,2019-05-31,0.105,0.020393,0.091461,3.991,99.658,95.900,98.405,3.483,0.508,10.476
AVH,2019-05-31,0.450,0.019414,0.075731,3.937,99.544,87.472,98.633,3.976,-0.039,-1.111
8CO,2019-05-31,0.155,0.017824,0.076189,3.486,99.431,88.383,44.761,3.636,-0.150,0.000
LTR,2019-05-31,0.105,0.018377,0.094203,3.442,99.317,97.608,98.975,3.816,-0.374,-4.762
WZR,2019-05-31,0.150,0.017412,0.100354,3.375,99.203,98.405,96.014,3.627,-0.252,-13.333
ISX,2019-05-31,0.620,0.015591,0.056436,3.123,99.089,65.945,85.649,3.180,-0.057,-6.452


In [3]:
# high price
df[df['start price']>1].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-31,6.760,0.021026,0.066211,4.132,99.886,78.246,21.526,4.705,-0.573,-3.846
TNY,2019-05-31,1.275,0.019912,0.106839,4.031,99.772,99.203,22.779,NaN,NaN,NaN
DUB,2019-05-31,1.240,0.013108,0.057602,2.720,98.747,73.121,66.173,2.199,0.521,19.758
Z1P,2019-05-31,3.200,0.012170,0.043293,2.473,98.405,52.961,87.813,2.412,0.061,-8.125
BUB,2019-05-31,1.245,0.011787,0.053695,2.314,98.178,68.565,95.216,2.362,-0.048,-1.606
CIA,2019-05-31,3.180,0.010987,0.039603,2.150,97.494,47.608,48.861,1.979,0.171,8.805
5GN,2019-05-31,1.335,0.010913,0.046615,2.119,97.267,60.592,53.531,2.579,-0.460,-1.124
JIN,2019-05-31,18.000,0.009599,0.027849,1.867,96.583,25.740,51.139,1.802,0.065,4.833
CRD,2019-05-31,1.690,0.009651,0.044229,1.849,96.355,58.428,47.494,1.750,0.099,1.775


In [4]:
# highest price
df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
MFG,2019-05-31,43.43,0.006196,0.018218,1.195,90.433,11.503,64.579,1.330,-0.135,-1.082
CUV,2019-05-31,34.49,0.006577,0.033461,1.301,92.141,46.014,27.563,1.339,-0.038,2.349
APX,2019-05-31,27.88,0.007906,0.031844,1.524,94.419,40.319,71.754,1.559,-0.035,5.882
APT,2019-05-31,24.04,0.006804,0.038466,1.276,91.686,52.392,85.421,1.505,-0.229,0.458
JIN,2019-05-31,18.00,0.009599,0.027849,1.867,96.583,25.740,51.139,1.802,0.065,4.833
IEL,2019-05-31,17.09,0.005415,0.025465,1.017,86.902,28.702,69.021,1.111,-0.094,2.926
BRG,2019-05-31,17.05,0.004989,0.023380,0.953,85.991,26.538,50.228,1.061,-0.108,-1.349
IVC,2019-05-31,15.63,0.004193,0.020111,0.814,82.232,18.451,62.073,0.948,-0.134,-2.175
LOV,2019-05-31,11.40,0.006490,0.031582,1.244,91.230,42.369,50.456,1.305,-0.061,-1.754


In [11]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

Low risk last week performance avg. = -0.9811%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
CHC,2019-05-31,10.340,0.003550,0.010535,0.680,78.018,2.164,82.802,0.709,-0.029,-2.224
AMC,2019-05-31,15.980,0.002279,0.007633,0.438,66.059,0.569,92.711,0.454,-0.016,-1.126
ASX,2019-05-31,75.280,0.002194,0.008543,0.408,64.180,1.367,52.050,0.526,-0.118,-1.514
SHL,2019-05-31,26.150,0.001944,0.008708,0.380,61.959,1.481,77.221,0.394,-0.014,-0.421
TCL,2019-05-31,13.930,0.001907,0.007810,0.369,61.219,1.025,94.989,NaN,NaN,NaN
CMW,2019-05-31,1.155,0.001808,0.007889,0.356,60.421,1.139,93.394,0.421,-0.065,-3.030
AST,2019-05-31,1.780,0.001532,0.007651,0.316,56.834,1.253,93.964,0.365,-0.049,-0.562
EGF,2019-05-31,1.650,0.001631,0.009270,0.315,56.720,2.278,2.620,NaN,NaN,NaN
STW,2019-05-31,59.890,0.001298,0.005541,0.265,53.360,0.114,27.790,NaN,NaN,NaN


In [13]:
# high volume
high_volume = df.sort_values(by='volume_rank', ascending=False).head(20).sort_values(by='return', ascending=False)
print('Highest volume last week performance avg. = %s%%'% week_return_avg(high_volume))
high_volume

Highest volume last week performance avg. = 1.263%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FEX,2019-05-31,0.105,0.020393,0.091461,3.991,99.658,95.900,98.405,3.483,0.508,10.476
AVH,2019-05-31,0.450,0.019414,0.075731,3.937,99.544,87.472,98.633,3.976,-0.039,-1.111
LTR,2019-05-31,0.105,0.018377,0.094203,3.442,99.317,97.608,98.975,3.816,-0.374,-4.762
FMG,2019-05-31,8.050,0.007215,0.027886,1.377,92.825,31.891,99.886,1.673,-0.296,-4.969
NEC,2019-05-31,2.080,0.004442,0.022715,0.872,84.055,24.943,98.747,0.885,-0.013,4.327
AVZ,2019-05-31,0.084,0.003524,0.071484,0.755,80.695,92.825,99.658,-0.419,1.174,42.857
BPT,2019-05-31,1.870,0.002938,0.025157,0.616,75.683,33.144,98.519,0.836,-0.220,-2.941
MGR,2019-05-31,3.030,0.003205,0.010838,0.601,74.943,3.189,99.431,0.690,-0.089,-2.970
MPL,2019-05-31,3.310,0.003024,0.017313,0.584,74.203,15.034,98.178,0.597,-0.013,1.813


In [14]:
# high return increase
df.sort_values(by='return_increase', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
LKE,2019-05-31,0.080,0.002988,0.074242,0.673,77.620,94.305,67.426,-1.052,1.725,43.750
AVZ,2019-05-31,0.084,0.003524,0.071484,0.755,80.695,92.825,99.658,-0.419,1.174,42.857
JHL,2019-05-31,0.059,0.011723,0.052420,2.318,98.292,67.312,7.403,1.247,1.071,42.373
CPV,2019-05-31,0.280,0.003062,0.055347,0.732,79.442,81.093,40.774,-0.059,0.791,16.071
KNO,2019-05-31,0.105,0.007222,0.057007,1.525,94.533,80.752,36.105,0.791,0.734,23.810
DEV,2019-05-31,0.068,0.004913,0.058240,1.023,87.244,84.055,48.747,0.322,0.701,26.471
ARU,2019-05-31,0.091,0.008047,0.058699,1.636,95.216,81.777,82.916,0.943,0.693,25.275
COB,2019-05-31,0.170,-0.000125,0.055141,0.036,34.112,83.941,45.444,-0.574,0.610,20.588
TDO,2019-05-31,0.115,0.004257,0.059747,0.859,83.827,84.624,23.804,0.267,0.592,20.870


In [15]:
# highest market cap.
highest_market_cap = [
    'BHP', 'RIO', 'CBA', 'WBC', 'CSL', 'ANZ', 'NAB', 'MQG', 'WOW', 'WES',
    'TLS', 'WPL'
    'S32', 'FMG', 'BXB', 'NCM', 'IAG', 'SUN', 'AMC', 'QBE', 'CIM', 'ALL',
    'AGL', 'ASX', 'RHC', 'OSH', 'SHL', 'TWE', 'REA', 'COH'
]
asx_20 = get_asx_20_list()
df[df.index.isin(asx_20)].sort_values(by='return', ascending=False).head(20)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
RIO,2019-05-31,100.30,0.002742,0.014172,0.553,72.722,9.681,83.485,0.648,-0.095,-2.712
TLS,2019-05-31,3.65,0.002560,0.012251,0.495,69.932,5.695,100.000,0.476,0.019,3.014
AMC,2019-05-31,15.98,0.002279,0.007633,0.438,66.059,0.569,92.711,0.454,-0.016,-1.126
BXB,2019-05-31,12.16,0.001909,0.009773,0.370,61.446,2.506,92.938,0.444,-0.074,-0.247
TCL,2019-05-31,13.93,0.001907,0.007810,0.369,61.219,1.025,94.989,NaN,NaN,NaN
WES,2019-05-31,37.05,0.001779,0.012437,0.343,59.112,8.200,87.927,0.430,-0.087,-2.969
ANZ,2019-05-31,28.01,0.001605,0.014416,0.308,56.321,12.187,95.558,0.394,-0.086,0.678
IAG,2019-05-31,7.65,0.001322,0.010963,0.272,53.986,5.125,95.103,0.301,-0.029,-3.137
WPL,2019-05-31,35.42,0.001310,0.011268,0.258,52.449,5.467,87.130,0.350,-0.092,-1.652


In [16]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
LKE,2019-05-31,0.080,0.002988,0.074242,0.673,77.620,94.305,67.426,-1.052,1.725,43.750
AVZ,2019-05-31,0.084,0.003524,0.071484,0.755,80.695,92.825,99.658,-0.419,1.174,42.857
JHL,2019-05-31,0.059,0.011723,0.052420,2.318,98.292,67.312,7.403,1.247,1.071,42.373
DEV,2019-05-31,0.068,0.004913,0.058240,1.023,87.244,84.055,48.747,0.322,0.701,26.471
ARU,2019-05-31,0.091,0.008047,0.058699,1.636,95.216,81.777,82.916,0.943,0.693,25.275
KNO,2019-05-31,0.105,0.007222,0.057007,1.525,94.533,80.752,36.105,0.791,0.734,23.810
GGG,2019-05-31,0.130,0.008462,0.061919,1.643,95.330,85.080,83.599,1.170,0.473,23.077
LYC,2019-05-31,3.050,0.007458,0.049616,1.564,94.875,67.768,96.469,1.019,0.545,21.967
CVN,2019-05-31,0.540,0.005822,0.038221,1.127,89.522,53.189,92.141,0.598,0.529,21.296


## Thisk week recommendations

In [17]:
# recommendations
selection = [
    'ANO',  # high return & high risk
    'ISX',  # high return & high risk
    'WKT',  # high return & low price & media risk
    'Z1P',  # high return & media risk
    'AD8',  # low risk & high price
    'LTR',  # high return & high volume
    'DUB',  # high increase
    'ALC',  # high increase
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-31,6.760,0.021026,0.066211,4.132,99.886,78.246,21.526,4.705,-0.573,-3.846
LTR,2019-05-31,0.105,0.018377,0.094203,3.442,99.317,97.608,98.975,3.816,-0.374,-4.762
ISX,2019-05-31,0.620,0.015591,0.056436,3.123,99.089,65.945,85.649,3.180,-0.057,-6.452
DUB,2019-05-31,1.240,0.013108,0.057602,2.720,98.747,73.121,66.173,2.199,0.521,19.758
WKT,2019-05-31,0.355,0.013725,0.052205,2.672,98.633,63.212,70.843,2.707,-0.035,11.268
Z1P,2019-05-31,3.200,0.012170,0.043293,2.473,98.405,52.961,87.813,2.412,0.061,-8.125
ALC,2019-05-31,0.115,0.011006,0.056086,2.139,97.380,76.196,88.383,1.568,0.571,20.000
AD8,2019-05-31,7.700,0.007712,0.024081,1.531,94.704,22.210,37.699,1.440,0.091,7.532


## Last week review

In [19]:
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
last_week_df = df[df.index.isin(selection)]
last_week_df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-31,6.76,0.021026,0.066211,4.132,99.886,78.246,21.526,4.705,-0.573,-3.846
AVH,2019-05-31,0.45,0.019414,0.075731,3.937,99.544,87.472,98.633,3.976,-0.039,-1.111
Z1P,2019-05-31,3.20,0.012170,0.043293,2.473,98.405,52.961,87.813,2.412,0.061,-8.125
AGH,2019-05-31,0.74,0.010303,0.044561,2.012,97.039,57.289,58.314,2.032,-0.020,10.811
AD8,2019-05-31,7.70,0.007712,0.024081,1.531,94.704,22.210,37.699,1.440,0.091,7.532
FMG,2019-05-31,8.05,0.007215,0.027886,1.377,92.825,31.891,99.886,1.673,-0.296,-4.969
MFG,2019-05-31,43.43,0.006196,0.018218,1.195,90.433,11.503,64.579,1.330,-0.135,-1.082


In [20]:
week_return = round(sum(last_week_df['week_return'])/len(selection), 3)
win_rate = round(len(last_week_df[last_week_df['week_return'] >= 0]) / len(selection), 3)
print('Last week average return:', week_return, ' , win rate:', win_rate)

Last week average return: -0.113  , win rate: 0.286


## Signle performance review

In [2]:
df[df.index=='FCT']

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FCT,2019-05-31,0.29,10.2824,434748.020202,0.007419,0.053854,1.446,1.466,-0.02,-10.345


# Return & Risk Rank

In [21]:
df['rank'] = df['return_rank'] + (100 - df['risk_rank'])
top_df = df.sort_values(by='rank', ascending=False).head(20)
week_return = round(sum(top_df['week_return'].dropna())/len(top_df), 3)
print('Top rank last week average return:', week_return)
top_df

Top rank last week average return: 0.293


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
MFG,2019-05-31,43.43,0.006196,0.018218,1.195,90.433,11.503,64.579,1.330,-0.135,-1.082,178.930
CHC,2019-05-31,10.34,0.003550,0.010535,0.680,78.018,2.164,82.802,0.709,-0.029,-2.224,175.854
DDR,2019-05-31,5.14,0.006631,0.021177,1.297,92.027,17.312,32.688,1.264,0.033,0.389,174.715
AD8,2019-05-31,7.70,0.007712,0.024081,1.531,94.704,22.210,37.699,1.440,0.091,7.532,172.494
MGR,2019-05-31,3.03,0.003205,0.010838,0.601,74.943,3.189,99.431,0.690,-0.089,-2.970,171.754
JIN,2019-05-31,18.00,0.009599,0.027849,1.867,96.583,25.740,51.139,1.802,0.065,4.833,170.843
APE,2019-05-31,10.45,0.006103,0.022147,1.222,90.547,20.501,5.239,0.984,0.238,9.091,170.046
JBH,2019-05-31,28.12,0.003252,0.013682,0.646,76.879,8.314,70.159,0.614,0.032,0.711,168.565
MND,2019-05-31,19.07,0.003353,0.014638,0.647,76.993,9.795,52.961,0.736,-0.089,-1.888,167.198
